In [1]:
import pandas as pd
import numpy as np
import ray

from d10_code import f10_import_data as dfi
from d10_code import f20_filter_data as dff
from d10_code import f30_clean_data as dfc
from d10_code import f40_analyse_data as dfa


def define_pandas_options():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)

In [2]:
 # Set options
define_pandas_options()
options = {
    'weeks_to_import': 1,
    'players_to_import_by_id': [],
    'players_to_import_by_name': [],
    'games_to_import': [2018090600],
    'teams_to_import': [],
    'team_type_to_import': ['home', 'away', 'football'],
    'plays_to_import': [75, 146],
    'directions_to_select': ['left', 'right'],
    'routes_to_select': []
}

# Import generic data
source_folder = './d00_source_data/'
players_df = dfi.import_player_data(source_folder + 'players.csv')
games_df = dfi.import_game_data(source_folder + 'games.csv')
plays_df = dfi.import_play_data(source_folder + 'plays.csv')

# Filter data to match options
players_df = dff.filter_player_data(players_df, options)
player_ids = players_df['nflId'].unique().tolist()

games_df = dff.filter_game_data(games_df, options)
game_ids = games_df['gameId'].unique().tolist()

plays_df = dff.filter_play_data(plays_df, options, game_ids)

# Clean data
players_df = dfc.clean_player_data(players_df)
games_df = dfc.clean_game_data(games_df)
plays_df = dfc.clean_play_data(plays_df)
play_ids = plays_df['gpid'].unique().tolist()

Reading in player data
Reading in game data
Reading in play data
Filtering player data
Removed 0 player entries.
Filtering game data
Removed 1512 game entries.
Filtering play data
Removed 519399 play-by-play entries.


In [3]:
tdf = dfa.analyse_play('2018090600-75', plays_df, games_df, players_df, source_folder)

away
Reading in tracking data for 2018090600 - 75 (Week: 1)
Filtering tracking data for 2018090600 - 75
                              time     s     a   dis event    nflId  \
0 2018-09-07 01:07:14.599000+00:00  0.00  0.01  0.02  None      310   
1 2018-09-07 01:07:14.599000+00:00  0.01  0.01  0.01  None    79848   
2 2018-09-07 01:07:14.599000+00:00  0.02  0.03  0.01  None  2495454   
3 2018-09-07 01:07:14.599000+00:00  0.09  0.42  0.01  None  2495613   
4 2018-09-07 01:07:14.599000+00:00  0.00  0.00  0.00  None  2533040   

       displayName  jerseyNumber position  frameId  team      gameId  playId  \
0        Matt Ryan             2       QB        1  away  2018090600      75   
1  Malcolm Jenkins            27       SS        1  home  2018090600      75   
2      Julio Jones            11       WR        1  away  2018090600      75   
3     Corey Graham            24       FS        1  home  2018090600      75   
4     Mohamed Sanu            12       WR        1  away  2018090600 

In [10]:
tdf.head()

,time,s,a,dis,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route,pos,teamType,o_vec,dir_vec,dx,dy
0,2018-09-07 01:07:14.599000+00:00,0.00,0.01,0.02,None,310,Matt Ryan,2,QB,1,away,2018090600,75,left,NaN,"[28.269999999999996, 26.629999999999995]",offense,"[0.9422329656752682, -0.33495826366099557]","[0.8740267478587447, 0.48587780771246014]",0.874027,0.485878
1,2018-09-07 01:07:14.599000+00:00,0.01,0.01,0.01,None,79848,Malcolm Jenkins,27,SS,1,home,2018090600,75,left,NaN,"[31.11, 16.83]",defense,"[-0.9630216286870437, 0.26942409446995247]","[-0.918170014691267, -0.3961866026531418]",-0.918170,-0.396187
2,2018-09-07 01:07:14.599000+00:00,0.02,0.03,0.01,None,2495454,Julio Jones,11,WR,1,away,2018090600,75,left,HITCH,"[28.650000000000006, 9.14]",offense,"[0.936977446144687, -0.34938984732842515]","[-0.2900341409153701, -0.9570162992883053]",-0.290034,-0.957016
3,2018-09-07 01:07:14.599000+00:00,0.09,0.42,0.01,None,2495613,Corey Graham,24,FS,1,home,2018090600,75,left,NaN,"[33.69, 31.289999999999996]",defense,"[-0.9404068845995315, -0.34005130700793285]","[-0.19235069571537686, 0.9813262504681155]",-0.192351,0.981326
4,2018-09-07 01:07:14.599000+00:00,0.00,0.00,0.00,None,2533040,Mohamed Sanu,12,WR,1,away,2018090600,75,left,HITCH,"[29.22, 17.15]",offense,"[0.9767097418432024, 0.21456486242762357]","[0.2414138168065054, 0.9704222632724968]",0.241414,0.970422


In [12]:
t_df = pd.DataFrame()